# Generates the mt5-small train embeddings

In [ ]:
pip install --user torch

You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --user datasets

You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --user transformers

You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --user sentence_transformers

You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --user fsspec

You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sentence_transformers import SentenceTransformer, models, InputExample, losses
from sentence_transformers.evaluation import LabelAccuracyEvaluator, SentenceEvaluator
from sentence_transformers.util import batch_to_device
from torch.utils.data import DataLoader
from transformers import MT5EncoderModel, AutoConfig,T5Tokenizer
from transformers import pipeline, AutoTokenizer

In [ ]:
import argparse

from sentence_transformers import SentenceTransformer, models
from tqdm import tqdm
import datasets

import torch
from transformers import TranslationPipeline, FeatureExtractionPipeline, MT5EncoderModel, AutoConfig
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers.pipelines.pt_utils import KeyDataset

#from models.model_mT5_contrastive import mT5Classifier

In [ ]:
import fsspec

In [ ]:
import os
import csv
import json
import logging
import math
from typing import List, Dict, Tuple, Callable, Iterable, Optional, Any
from tqdm import tqdm
import torch
import torch.nn as nn
import pandas as pd
import pickle
import numpy as np
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from numpy import save
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
import datasets

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization

from sklearn.model_selection import train_test_split

In [ ]:
import datasets

import torch
from transformers import TranslationPipeline, FeatureExtractionPipeline, MT5EncoderModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers.pipelines.pt_utils import KeyDataset

In [ ]:
def embed_sentence(in_path: str, out_path: str, finetuned: bool=False, batch_size:int=32):
    # tokenizer = AutoTokenizer.from_pretrained("google/mt5-small", model_max_length=512)
    # model = MT5EncoderModel.from_pretrained("google/mt5-small")
    # pipe = FeatureExtractionPipeline(model, tokenizer=tokenizer, device=-1, batch_size=batch_size)
    if finetuned:
        model = mT5Classifier(load_model_path="dump/mt5_sup_chi")
    else:
        model_name = "google/mt5-small"
        base_model = models.Transformer(model_name)
        # sentence bert repo does not current support mT5 yet (only T5 ironically)
        MT5EncoderModel._keys_to_ignore_on_load_unexpected = ["decoder.*"]
        mT5_config = AutoConfig.from_pretrained(model_name)
        base_model.auto_model = MT5EncoderModel.from_pretrained("google/mt5-small", config=mT5_config)
        pooling_model = models.Pooling(base_model.get_word_embedding_dimension())
        model = SentenceTransformer(modules=[base_model, pooling_model])

    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    kwargs = {'truncation': True, 'max_length': 512, 'return_text': True, "clean_up_tokenization_spaces": True}
    if in_path.endswith(".txt"):
        dataset = datasets.load_dataset('text', data_files={'train': [in_path]})["train"]
    elif in_path.endswith(".csv"):
        dataset = datasets.load_dataset('csv', data_files={'train': [in_path]})["train"]
        dataset = dataset.rename_column("Text", "text")
    elif ".jsonl" in in_path:
        dataset = datasets.load_dataset('text', data_files={'train': [in_path]})["train"]
        dataset = dataset.map(lambda example: {"content": json.loads(example["text"])["text"]}, remove_columns=["text"])
        dataset = dataset.rename_column("content", "text")
    else:
        raise NotImplementedError()

    # pbar = tqdm(total=len(dataset))
    # embeddings = np.zeros([len(dataset), model.config.hidden_size if not finetuned else 256])
    # for i, out in enumerate(pipe(KeyDataset(dataset, "text"), **kwargs)):
        # embeddings[i] = np.array(out[0][:512]).mean(1)
        # pbar.update(1)
    embeddings = model.encode(dataset["text"], batch_size=batch_size, show_progress_bar=True, device="cpu")
    np.save(out_path, embeddings)


In [ ]:
embed_sentence('Data/Train.csv','Data/embeddings_chichewa_512_final.npy', finetuned=False, batch_size=2)

Some weights of the model checkpoint at google/mt5-small were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at google/mt5-small were not used when initializing MT5EncoderModel: ['lm_head.weight']
- This IS expected if you are initializing MT5EncoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5EncoderModel from the 

  0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/718 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
embds_ = np.load('Data/embeddings_chichewa_512_final.npy')

In [ ]:
embds_.shape

(1436, 512)

In [ ]:
embds_[0]

array([ 1.40220616e-02, -9.88253355e-02,  1.52107710e-02, -1.47006691e-01,
       -4.01516482e-02,  5.06973565e-02,  2.43828192e-01,  5.38479164e-02,
        7.54577667e-02,  4.41238023e-02,  1.26080036e-01,  1.44053036e-02,
       -5.92401028e-02, -9.04205739e-02,  1.84907749e-01, -1.63099185e-01,
       -1.59675062e-01, -6.66518733e-02,  5.37426444e-03,  2.00472072e-01,
        1.56612247e-01,  1.12344936e-01, -3.85461785e-02,  3.73733677e-02,
       -1.91163063e-01, -7.08642080e-02, -6.82506204e-01, -2.38565840e-02,
        2.04631448e-01,  2.16051489e-01,  1.56753927e-01, -5.71367145e-03,
       -8.91433582e-02,  4.53606732e-02, -9.36719179e-02, -1.96273588e-02,
        7.95600340e-02,  1.16714016e-01,  2.70994604e-01, -2.00763255e-01,
        9.55745503e-02,  4.81148325e-02, -5.81331030e-02,  1.46594629e-01,
        9.41835791e-02, -1.52396500e-01,  9.13952067e-02,  5.71678095e-02,
        7.26696178e-02, -1.15161948e-01,  1.28319478e-02,  6.14085980e-03,
        1.04273647e-01,  

In [ ]:
embed_sentence('Data/train_google_translated.csv','Data/embeddings_english_512_final.npy', finetuned=False, batch_size=2)

Some weights of the model checkpoint at google/mt5-small were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at google/mt5-small were not used when initializing MT5EncoderModel: ['lm_head.weight']
- This IS expected if you are initializing MT5EncoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5EncoderModel from the 

  0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/718 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
embds_english = np.load('Data/embeddings_english_512_final.npy')

In [ ]:
embds_english.shape

(1436, 512)

In [ ]:
embds_english[0]

array([ 1.04368024e-01,  4.51874593e-03,  7.95077439e-03, -3.81290205e-02,
        5.19804657e-04,  8.02169591e-02,  2.59716392e-01, -4.28596660e-02,
        6.78706914e-02, -8.73456001e-02,  1.14334069e-01, -6.46002442e-02,
       -1.89171106e-01, -2.52639540e-02,  2.42037550e-01, -1.83425695e-01,
        6.74404055e-02, -4.32823114e-02, -2.42234413e-02,  2.34039143e-01,
        1.38632536e-01,  1.94235161e-01, -4.10279892e-02,  6.09163754e-02,
        6.62765130e-02, -7.28116557e-02, -6.90937221e-01, -6.75147101e-02,
       -1.49431929e-01,  1.42560661e-01,  1.33726031e-01, -5.46459481e-02,
       -1.19197620e-02,  5.87540194e-02, -1.00289188e-01, -1.72122687e-01,
       -6.36342987e-02,  1.46076456e-01,  4.47292686e-01, -2.13453367e-01,
        5.86086065e-02,  4.20327447e-02,  8.06729198e-02,  2.50032187e-01,
        8.88898149e-02,  9.25356373e-02,  6.12092391e-02, -5.73230535e-02,
        8.52963403e-02, -2.90111452e-02,  2.78195441e-02, -9.11372751e-02,
        1.19542629e-01,  